In [1]:
import requests
import json
import time
import re
import datetime
import calendar
import dateutil.parser as parser
from dateutil.relativedelta import relativedelta
from datetime import datetime, timezone
import yaml
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import plotly.graph_objects as go
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist
from sklearn.neighbors import NearestNeighbors
from yellowbrick.cluster import KElbowVisualizer, SilhouetteVisualizer
import os
import kaleido

In [2]:
def convert_date(utc_time): 
    parsed_date = parser.parse(utc_time)
    var_date=parsed_date.date()
    var_time=parsed_date.time()
    var_weekday=parsed_date.weekday()
    var_weekday_name=calendar.day_name[parsed_date.weekday()]
    return var_date, var_time, var_weekday ,var_weekday_name

In [3]:
def eda(data):
    output=[]
    for col in data.columns:
        duplicatedvalue = data[col].duplicated().sum()
        duplicatedrows = data.duplicated().sum()
        missingvalue = np.sum(pd.isna(data[col]))
        uniquevalue = data[col].nunique()
        datatype = str(data[col].dtype)
        
        output.append([col, duplicatedvalue, duplicatedrows, missingvalue, uniquevalue, datatype])
        
    output = pd.DataFrame(output) 
    output.columns = ['Features', 'Duplicated Values', 'Duplicated Rows', 'Missing Values', 'Unique Values', 'Data Type']
    display(output)

In [4]:
def perc_on_bar(plot, feature):
    total = len(feature)
    for p in ax.patches:
        percentage = "{:.1f}%".format(100 * p.get_height() / total)
        x = p.get_x() + p.get_width() / 2 - 0.05
        y = p.get_y() + p.get_height()
        ax.annotate(percentage, (x, y), size=12)
    plt.show()

In [5]:
def viz(data, candle_no):
    fig = go.Figure(data=[go.Candlestick(x=data['UTC_Time'],
                open=data['Open'],
                high=data['High'],
                low=data['Low'],
                close=data['Close'])])

    fig.update_layout(xaxis_rangeslider_visible=False,
                      title=pair + " " + timeframe,
                      title_font_color="blue",
                      title_font_size = 30)
    
    fig.update_xaxes(rangebreaks=[dict(bounds=["sat", "mon"])])
    
    fig.write_image(path + "/" + candle_no + "_chart.png")
    
    fig.show()

In [6]:
def market_order(instrument, units, take_profit, stop_loss):
    login_request_body = {
        "order": {
            "type": "MARKET",
            "instrument": instrument,
            "units": units,
            "timeInForce": "IOC",
            "positionFill": "DEFAULT",
            "takeProfitOnFill": {
                "price": take_profit
            },
            "stopLossOnFill": {
                "price": stop_loss
            }
        }
    }
    response = requests.post(provider_api_url, data=json.dumps(login_request_body),
                             headers=request_headers,
                             verify=False)
    response
    response.status_code

### The Configs for Run:

In [7]:
with open ('config.yml') as ymlfile:
    cfg = yaml.safe_load(ymlfile)
    oanda_api_key = cfg['creds']['oanda_api']
    account_number = cfg['creds']['account_number'] 

### <font color='red'>Currency Pair</font>

In [8]:
Load_10K_Records=True

currency_pairs = ['EUR_USD','EUR_GBP','EUR_NZD','EUR_AUD','EUR_CHF','EUR_CAD',
                  'GBP_USD','GBP_CHF','GBP_NZD','GBP_AUD','GBP_CAD','AUD_USD',
                  'AUD_CAD','AUD_CHF','AUD_NZD','NZD_USD','NZD_CHF','NZD_CAD',
                  'USD_CAD','USD_CHF','CAD_CHF']

currency_pairs = ["EUR_USD"]


timeframe = "H4"
#D #H1 #H4 M30
# https://developer.oanda.com/rest-live-v20/instrument-df/#CandlestickGranularity
price_char = "M"
#M(midpoint candles) #B(bid candles) #A(ask candles) #BA
price_com = "mid"
#mid #bid #ask

# def of OANDA request variable
provider_api_url = 'https://api-fxpractice.oanda.com/v3/accounts/101-001-16477519-001/orders'
request_headers = {
    "Authorization": oanda_api_key,
    "Accept-Datetime-Format": "RFC3339",
    "Connection": "Keep-Alive",
    "Content-Type": "application/json;charset=UTF-8"
}


In [9]:
provider_authorization = 'Bearer {0}'.format(oanda_api_key)

headers = {
    'Content-Type': 'application/json',
    'Authorization': provider_authorization,
}

### Logging

In [10]:
now = datetime.now()
for pair in currency_pairs:
    Log_Folder = now.strftime("%d-%m-%Y_%I-%M_%p")
    path = os.path.join(Log_Folder+"_"+pair)
    os.mkdir(path)

# Get Candlesticks Data

### Pricing & Spread Cost

In [11]:
for pair in currency_pairs:
    pricing_params = (
        ('instruments', pair),
    )
    response = requests.get('https://api-fxpractice.oanda.com/v3/accounts/{}/pricing'.format(account_number),
                            headers=headers,
                            params=pricing_params).json()

In [12]:
time = response['time']
ask = response['prices'][0]['closeoutAsk']
bid = response['prices'][0]['closeoutBid']
print ('Date:', time, 'Ask:', ask, 'Bid:', bid)

Date: 2021-09-26T22:46:49.410051834Z Ask: 1.17211 Bid: 1.17207


### Open Positions

In [13]:
response = requests.get('https://api-fxpractice.oanda.com/v3/accounts/{}/openPositions'.format(account_number),
                            headers=headers,
                            params=pricing_params).json()
response['positions']

[{'instrument': 'EUR_USD',
  'long': {'units': '0',
   'pl': '-44894.1846',
   'resettablePL': '-44894.1846',
   'financing': '-354.9302',
   'dividendAdjustment': '0.0000',
   'guaranteedExecutionFees': '0.0000',
   'unrealizedPL': '0.0000'},
  'short': {'units': '-1000000',
   'averagePrice': '1.16879',
   'pl': '5318.9488',
   'resettablePL': '5318.9488',
   'financing': '-53.9916',
   'dividendAdjustment': '0.0000',
   'guaranteedExecutionFees': '0.0000',
   'tradeIDs': ['5851'],
   'unrealizedPL': '-3320.0000'},
  'pl': '-39575.2358',
  'resettablePL': '-39575.2358',
  'financing': '-408.9218',
  'commission': '1700.0200',
  'dividendAdjustment': '0.0000',
  'guaranteedExecutionFees': '0.0000',
  'unrealizedPL': '-3320.0000',
  'marginUsed': '23441.8000'}]

###  Candlestick Data

In [14]:
params_count = (
    ('price', price_char),
    ('count', '5000'),
    ('granularity', timeframe),
)


In [15]:
for pair in currency_pairs:
    first_response = requests.get('https://api-fxpractice.oanda.com/v3/instruments/{}/candles'.format(pair), 
                            headers=headers,
                            params=params_count).json()

In [16]:
if Load_10K_Records:
    datetime_object = parser.parse(first_response['candles'][0]['time'])
    date= datetime_object - relativedelta(years=3)  
    from_date = date.replace(tzinfo=timezone.utc).timestamp()
    params_date = (
        ('count', '5000'),
        ('price', price_char),
        ('from', from_date),
        ('granularity', timeframe),)

    second_response = requests.get('https://api-fxpractice.oanda.com/v3/instruments/{}/candles'.format(pair),
                                   headers=headers,
                                   params=params_date).json()
            
    first_response= first_response['candles']  
    second_response= second_response['candles']
    second_response.extend(first_response)
    response=second_response
else:
    response=first_response['candles']

In [17]:
filename = "{}_{}.csv".format(pair, timeframe)
output = []
all_candlesticks = response

for i in range (len(all_candlesticks)):
    result= (convert_date(response[i]['time']))
    output.append([(result[0]),(result[1]),(result[2]),(result[3]),
                    response[i]['time'],
                    response[i]['volume'], 
                    response[i][price_com]['o'],
                    response[i][price_com]['h'],
                    response[i][price_com]['l'],
                    response[i][price_com]['c']])
    
output = pd.DataFrame(output)
output.columns = ['Date', 'Time', 'Weekday','Weekday_Name','UTC_Time', 'Volume', 'Open', 'High', 'Low', 'Close']
data = output.to_csv(filename, header = True, index = False)
data = pd.read_csv(filename)

In [18]:
data.head()

,Date,Time,Weekday,Weekday_Name,UTC_Time,Volume,Open,High,Low,Close
0,2015-07-09,21:00:00,3,Thursday,2015-07-09T21:00:00.000000000Z,3928,1.10354,1.10833,1.10312,1.10731
1,2015-07-10,01:00:00,4,Friday,2015-07-10T01:00:00.000000000Z,3573,1.10733,1.11040,1.10718,1.10970
2,2015-07-10,05:00:00,4,Friday,2015-07-10T05:00:00.000000000Z,12003,1.10968,1.11340,1.10786,1.11136
3,2015-07-10,09:00:00,4,Friday,2015-07-10T09:00:00.000000000Z,13740,1.11133,1.12159,1.11092,1.11974
4,2015-07-10,13:00:00,4,Friday,2015-07-10T13:00:00.000000000Z,11703,1.11971,1.12078,1.11262,1.11285


In [19]:
data.tail()

,Date,Time,Weekday,Weekday_Name,UTC_Time,Volume,Open,High,Low,Close
9995,2021-09-24,05:00:00,4,Friday,2021-09-24T05:00:00.000000000Z,13270,1.17373,1.17416,1.17309,1.17359
9996,2021-09-24,09:00:00,4,Friday,2021-09-24T09:00:00.000000000Z,15460,1.17360,1.17382,1.17066,1.17066
9997,2021-09-24,13:00:00,4,Friday,2021-09-24T13:00:00.000000000Z,20795,1.17066,1.17255,1.17008,1.17162
9998,2021-09-24,17:00:00,4,Friday,2021-09-24T17:00:00.000000000Z,5123,1.17163,1.17227,1.17132,1.17206
9999,2021-09-26,21:00:00,6,Sunday,2021-09-26T21:00:00.000000000Z,9687,1.17281,1.17281,1.17157,1.17209


In [20]:
data = data.drop_duplicates()

In [21]:
data = data.to_csv(filename, header = True, index = False)

In [22]:
data = pd.read_csv(filename)

In [23]:
data.shape

(9659, 10)

# <font color='red'>CANDLE INDEX NUMBER</font>

In [24]:
candle_no = len(data) - 2
candle_no

9657

# Feature Engineering

### CandleStick Data

In [25]:
data['O-H'] = data['Open'] - data['High']
data['O-L'] = data['Open'] - data['Low']
data['O-C'] = data['Open'] - data['Close']
data['H-L'] = data['High'] - data['Low']
data['H-C'] = data['High'] - data['Close']
data['L-C'] = data['Low'] - data['Close']

In [26]:
data['Direction'] = data['O-C'].apply(lambda x: 1 if x<0 else 0)

In [27]:
data = data.to_csv(filename, header = True, index = False)

In [28]:
data = pd.read_csv(filename)

In [29]:
data.head()

,Date,Time,Weekday,Weekday_Name,UTC_Time,Volume,Open,High,Low,Close,O-H,O-L,O-C,H-L,H-C,L-C,Direction
0,2015-07-09,21:00:00,3,Thursday,2015-07-09T21:00:00.000000000Z,3928,1.10354,1.10833,1.10312,1.10731,-0.00479,0.00042,-0.00377,0.00521,0.00102,-0.00419,1
1,2015-07-10,01:00:00,4,Friday,2015-07-10T01:00:00.000000000Z,3573,1.10733,1.11040,1.10718,1.10970,-0.00307,0.00015,-0.00237,0.00322,0.00070,-0.00252,1
2,2015-07-10,05:00:00,4,Friday,2015-07-10T05:00:00.000000000Z,12003,1.10968,1.11340,1.10786,1.11136,-0.00372,0.00182,-0.00168,0.00554,0.00204,-0.00350,1
3,2015-07-10,09:00:00,4,Friday,2015-07-10T09:00:00.000000000Z,13740,1.11133,1.12159,1.11092,1.11974,-0.01026,0.00041,-0.00841,0.01067,0.00185,-0.00882,1
4,2015-07-10,13:00:00,4,Friday,2015-07-10T13:00:00.000000000Z,11703,1.11971,1.12078,1.11262,1.11285,-0.00107,0.00709,0.00686,0.00816,0.00793,-0.00023,0


# Modeling

In [30]:
data = pd.read_csv(filename)

In [31]:
data = data.drop(columns=['Volume','Weekday','Date','Time',
                          'Weekday_Name','UTC_Time','Direction',
                          'Open', 'High', 'Low', 'Close'])

#'Volume', 'Direction'

### Scaling using Standard Scaler

In [32]:
def find_k_similar_candles(candle_id, dataset, k=6):
    indices=[]
    distances = []
    output = []
    model_knn = NearestNeighbors(metric = 'euclidean', algorithm = 'brute') 
    model_knn.fit(dataset)
    
    #metric = 'euclidean' or 'cosine'
    
    distances, indices = model_knn.kneighbors(dataset.iloc[candle_id,:].values.reshape(1,-1),
                                              n_neighbors = k)

    for i in range(0,len(distances.flatten())):
        if i==0:
            display (pd.DataFrame(data.iloc[candle_id]).transpose())
            #print("Recommendation for {0}:\n".format(eurusd_data.index[candle_id]))
        else:
            #print("{0}: {1}, with distance of {2}".format(i,
            #                                               dataset.index[indices.flatten()[i]],
            #                                               distances.flatten()[i]))
            
            output.append ([dataset.index[indices.flatten()[i]],
                            distances.flatten()[i],
                            dataset.iloc[indices.flatten()[i]]['O-H'],
                            dataset.iloc[indices.flatten()[i]]['O-L'],
                            dataset.iloc[indices.flatten()[i]]['O-C'],
                            dataset.iloc[indices.flatten()[i]]['H-L'],
                            dataset.iloc[indices.flatten()[i]]['H-C'],
                            dataset.iloc[indices.flatten()[i]]['L-C']])
    
    output = pd.DataFrame(output)
    output.columns = ['Indice','Distance','O-H','O-L','O-C','H-L','H-C','L-C']
    display (output)
    
    return indices, distances

### Top 5 Similar Candlesticks

In [33]:
indices, distances = find_k_similar_candles (candle_no,data)

,O-H,O-L,O-C,H-L,H-C,L-C
9657,-0.00064,0.00031,-0.00043,0.00095,0.00021,-0.00074


,Indice,Distance,O-H,O-L,O-C,H-L,H-C,L-C
0,5907,0.000100,-0.00060,0.00032,-0.00046,0.00092,0.00014,-0.00078
1,4905,0.000118,-0.00072,0.00025,-0.00048,0.00097,0.00024,-0.00073
2,3652,0.000131,-0.00064,0.00038,-0.00037,0.00102,0.00027,-0.00075
3,7030,0.000141,-0.00056,0.00038,-0.00034,0.00094,0.00022,-0.00072
4,5906,0.000142,-0.00060,0.00039,-0.00044,0.00099,0.00016,-0.00083


In [34]:
indices = indices[0:1][0]

In [35]:
indices

array([9657, 5907, 4905, 3652, 7030, 5906], dtype=int64)

### Currnet Market/Candlestick

In [ ]:
closed_candle = "currnet_market_data.csv"
data = pd.read_csv(filename)
data = data.iloc[candle_no-60:candle_no+1]
data.to_csv(path + "/" + closed_candle, header = True, index = False)
viz(data, "current_market")

### Recommendations

In [ ]:
for indice in indices[1:6]:
    recommendation_log = "{}_data.csv".format(indice)
    data = pd.read_csv(filename)
    data = data.iloc[indice:indice+50]
    data.to_csv(path + "/" + recommendation_log, header = True, index = False)
    print ('Neighbor:', indice, '|', '10K Records:', Load_10K_Records)
    viz(data, indice.astype(str))